# 1 Monte um passo a passo para o Bagging

## Bootstrap

Método de obtenção de datasets por amostragem com reposição de um dataset original.

## Model

Treinamento dos modelos base learnings usando os diferentes datasets geradas pelo bootstrap

## Aggregate

Sumarização dos resultados obtidos nos modelos.

---

# 2 Explique com as suas palavras o Bagging

## Bootstrap

Bootstrap é um método de obtenção de datasets que são amostras de um dataset original. Um dataset será gerado com o mesmo número de instâncias do original. Cada uma dessas entradas desse novo dataset será obtida sorteando-se uma das instâncias originais. Em cada sorteio, todas as instâncias têm a mesma probabilidade de serem escolhidas, sendo portanto uma amostragem randômica com reposição.

## Model

Cada uma das datasets será usada para a determinação de um modelo, por treinamento com esse dataset. Esses modelos são chamdos de base learnings.

## Aggregate

De cada modelo, será obtido os resultados de predição. 

- Caso seja um problema de classificação, o resultado será a classe que mais se repetil nas previsões realizadas pelos modelos.
- Caso seja um problema de regressão, será usada a média aritmética dos valores obtidos pelas previsões dos modelos.

---

# 3 Implementação em python do Bagging

In [1]:
# Importação das bibliotecas
import pandas as pd
import numpy as np

# Modelagem
import statsmodels.formula.api as smf

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Instanciamento do dataset original dontendo dados randômicos
num_instancias = 100
data = {
    "feature_1": np.random.rand(num_instancias),
    "feature_2": np.random.randint(1, 10, num_instancias),
    "feature_3": np.random.randn(1, num_instancias)[0],
    "feature_4": np.random.randint(10, 100, num_instancias),
    "target": np.random.randint(0, 2, num_instancias),
}
df = pd.DataFrame(data)
df = df.astype("float64")
df.head()

,feature_1,feature_2,feature_3,feature_4,target
0,0.059351,5.0,1.412055,31.0,1.0
1,0.944962,7.0,-0.443070,75.0,0.0
2,0.653520,2.0,0.181604,11.0,1.0
3,0.542496,7.0,-0.294067,67.0,0.0
4,0.813139,2.0,-0.571971,92.0,1.0


In [3]:
# Verificando as variáveis do dataset original
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   feature_1  100 non-null    float64
 1   feature_2  100 non-null    float64
 2   feature_3  100 non-null    float64
 3   feature_4  100 non-null    float64
 4   target     100 non-null    float64
dtypes: float64(5)
memory usage: 4.0 KB


## Bootstrap

In [4]:
def bootstrap(dataset: pd.DataFrame) -> pd.DataFrame:
    '''
    Função que recebe o dataset original e retorna um novo por bootstrap
    :param dataset (pd.DataFrame): Original
    :return new_dataset (pd.DataFrame): Bootstrap
    '''
    dataset = df.copy()
    new_dataset = pd.DataFrame(columns=dataset.columns)
    sample_enter = dataset.sample(1)

    for _ in range(len(dataset)):
        new_dataset = pd.concat((new_dataset, sample_enter))
        sample_enter = dataset.sample(1)
    new_dataset = new_dataset.astype("float64")
    return new_dataset

In [5]:
# Instanciando uma lista contendo todos os datasets obtidos por bootstrap

# Serão obtidos 20 datasets
entradas = 20

# Index para os datasets
set_teste = {i for i in range(entradas)}

c = True
while c:

    # Instanciando uma lista com os datasets
    df_bootstrap = [bootstrap(df) for _ in range(entradas)]

    # Instancia um array com os index contidos em cada um dos datasets
    df_index = np.array([list(df_bootstrap[df_test].index) for df_test in range(entradas)])
    try:

        # Verifica se o index do dataset padrão está presente em cada um dos datset obtidos por bootstrap
        save_index = [i for i in df.index.values if set(np.where(df_index == i)[0]) == set_teste][0]

        # Caso o index esteja presente em todos, encerra o while
        c = False
        print(f"Index comum: {save_index}")
    except:
        print("Sem index comum")

save_index

Sem index comum
Sem index comum
Sem index comum
Sem index comum
Sem index comum
Sem index comum
Sem index comum
Sem index comum
Sem index comum
Sem index comum
Sem index comum
Sem index comum
Sem index comum
Sem index comum
Sem index comum
Index comum: 70


70

## Model

In [6]:
# Determinação do modelo de regressão logistica usando o primeiro dataset obtido por bootstrap
modelo = "target ~ feature_1 + feature_2 + feature_3 + feature_4"

reglog = smf.logit(
    modelo,
    data=df_bootstrap[0]
).fit(method='bfgs')

reglog.summary()

Optimization terminated successfully.
         Current function value: 0.670015
         Iterations: 23
         Function evaluations: 28
         Gradient evaluations: 28


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 target   No. Observations:                  100
Model:                          Logit   Df Residuals:                       95
Method:                           MLE   Df Model:                            4
Date:                Fri, 28 Jun 2024   Pseudo R-squ.:                 0.03225
Time:                        13:12:33   Log-Likelihood:                -67.002
converged:                       True   LL-Null:                       -69.235
Covariance Type:            nonrobust   LLR p-value:                    0.3466
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0331      0.831     -0.040      0.968      -1.661       1.595
feature_1      0.2967      0.712      0.417      0.677      -1.100       1.693
feature_2     -0.1301      0.091     -1.433      0.152      -0.308       0.048
feature_3      0.1847      0.241      0.767      0.443      -0.288       0.657
feature_4      0.0081      0.009      0.922      0.356      -0.009       0.025
==============================================================================
"""

In [7]:
# Faz a predição do target e comparar com o dado original
pred = reglog.predict(df.loc[save_index, :])
df['pred'] = (pred > 0.5).astype("int64")
df.dropna()

,feature_1,feature_2,feature_3,feature_4,target,pred
70,0.362618,2.0,-0.967787,68.0,0.0,1.0


In [8]:
def base_learnings(df_train: pd.DataFrame, save_index: int) -> dict:
    '''
    Função de obtenção dos Base learnings.
    Recebe uma DataFrame e determina a predição do target em uma instância específica
    :param df_trains (pd.DataFrame): Dataset para ser usado no treinamento
    :return (int): Dicionário com a predição dos valores obtidos no treinamento
    '''
    modelo = "target ~ feature_1 + feature_2 + feature_3 + feature_4"
    reg = smf.logit(
        modelo,
        data=df_train
    ).fit(method='bfgs').predict(df_train.loc[save_index, :])

    retorno = {
        "index": pred.index[0],
        "valor": pred.values[0],
        "target_bool": pred.values[0] > 0.5,
        "target_float": float(pred.values[0] > 0.5),
    }

    return retorno

In [9]:
# Testando a função de predição em um dataset bootstrap
base_learnings(df_train=df_bootstrap[0], save_index=save_index)

Optimization terminated successfully.
         Current function value: 0.670015
         Iterations: 23
         Function evaluations: 28
         Gradient evaluations: 28


{'index': 70,
 'valor': 0.545935516977216,
 'target_bool': True,
 'target_float': 1.0}

In [10]:
# Testando a função de predição em todos os datasets bootstrap
aggregat_list = [base_learnings(df_train=df, save_index=save_index)["target_float"] for df in df_bootstrap]
aggregat_list = np.array(aggregat_list)
aggregat_list

Optimization terminated successfully.
         Current function value: 0.670015
         Iterations: 23
         Function evaluations: 28
         Gradient evaluations: 28
Optimization terminated successfully.
         Current function value: 0.627919
         Iterations: 22
         Function evaluations: 25
         Gradient evaluations: 25
Optimization terminated successfully.
         Current function value: 0.578966
         Iterations: 21
         Function evaluations: 25
         Gradient evaluations: 25
Optimization terminated successfully.
         Current function value: 0.644088
         Iterations: 23
         Function evaluations: 26
         Gradient evaluations: 26
Optimization terminated successfully.
         Current function value: 0.646748
         Iterations: 23
         Function evaluations: 26
         Gradient evaluations: 26
Optimization terminated successfully.
         Current function value: 0.606163
         Iterations: 24
         Function evaluations: 27
  

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])

## Aggregation

In [11]:
# Compara os valores obtidos e retorna o mais frequente pelo método de agregação 
num_zeros = (np.array(aggregat_list) == 0.0).sum()
num_ones = (np.array(aggregat_list) != 0.0).sum()
if num_zeros > num_ones:
    aggregat = 0.0
    print(f"Aggregation = {aggregat}")
else:
    aggregat = 1.0
    print(f"Aggregation = {aggregat}")

Aggregation = 1.0
